<a href="https://colab.research.google.com/github/myazann/M3-Machine-Learning/blob/main/M3_Week3_Model_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!cp drive/MyDrive/utils.py .
!cp drive/MyDrive/mlp_MIT_8_scene.py .
!cp drive/MyDrive/patch_based_mlp_MIT_8_scene.py .

!cp -a drive/MyDrive/MIT_split .
!mkdir work

In [1]:
import os
import getpass

from utils import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import time

In [ ]:
work_dir = "work/" + str(time.time())
os.mkdir(work_dir)

IMG_SIZE = 64
BATCH_SIZE  = 16
#DATASET_DIR = '/home/mcv/datasets/MIT_split'
DATASET_DIR = 'MIT_split'
#MODEL_FNAME = '/home/group06/work/my_first_mlp.h5'
MODEL_FNAME = work_dir + '/model.h5'

if not os.path.exists(DATASET_DIR):
    print(Color.RED, 'ERROR: dataset directory ' + DATASET_DIR + ' do not exists!\n')

print('Building MLP model...\n')


#Build the Multi Layer Perceptron model
model = Sequential()
model.add(Reshape((IMG_SIZE*IMG_SIZE*3,), input_shape=(IMG_SIZE, IMG_SIZE, 3), name='first'))
model.add(Dense(units=2048, activation='leaky_relu',name='second'))
model.add(Dense(units=1024, activation='leaky_relu', name='third'))
model.add(Dense(units=512, activation='leaky_relu', name='fourth'))
model.add(Dense(units=8, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(),
              metrics=['accuracy'])

print(model.summary())
plot_model(model, to_file=work_dir + '/model.png', show_shapes=True, show_layer_names=True)

print('Done!\n')

if os.path.exists(MODEL_FNAME):
    print('WARNING: model file ' + MODEL_FNAME + ' exists and will be overwritten!\n')

print('Start training...\n')

# this is the dataset configuration we will use for training
# only rescaling
train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True)

# this is the dataset configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        DATASET_DIR+'/train',  # this is the target directory
        target_size=(IMG_SIZE, IMG_SIZE),  # all images will be resized to IMG_SIZExIMG_SIZE
        batch_size=BATCH_SIZE,
        classes = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding'],
        class_mode='categorical')  # since we use binary_crossentropy loss, we need categorical labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        DATASET_DIR+'/test',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        classes = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding'],
        class_mode='categorical')

history = model.fit_generator(
        train_generator,
        steps_per_epoch=1881 // BATCH_SIZE,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=807 // BATCH_SIZE)

print('Done!\n')
print('Saving the model into ' + MODEL_FNAME + ' \n')
model.save(MODEL_FNAME)  # always save your weights after training or during training
print('Done!\n')

  # summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig(work_dir + '/accuracy.jpg')
plt.close()
  # summarize history for loss

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig(work_dir + '/loss.jpg')



In [3]:
second_layer = Model(inputs=model.input, outputs=model.get_layer('second').output)
third_layer = Model(inputs=model.input, outputs=model.get_layer('third').output)
fourth_layer = Model(inputs=model.input, outputs=model.get_layer('fourth').output)

In [39]:
with open(DATASET_DIR + "/test_images_filenames.dat") as test_images_filenames:
  test_images_filenames = test_images_filenames.readlines()
  test_files = [k[k.find("MIT_split"):].strip()[:-1] for k in test_images_filenames if "MIT_split" in k]

second_layer_preds = []
third_layer_preds = []
fourth_layer_preds = []

for img in test_files:
  x = np.asarray(Image.open(img))
  x = np.expand_dims(cv2.resize(x, (IMG_SIZE, IMG_SIZE)), axis=0)/255.0

  second_layer_preds.append(second_layer.predict(x))
  third_layer_preds.append(third_layer.predict(x))
  fourth_layer_preds.append(fourth_layer.predict(x))
  
second_layer_preds = np.vstack(second_layer_preds)
third_layer_preds = np.vstack(third_layer_preds)
fourth_layer_preds = np.vstack(fourth_layer_preds)

In [49]:
import pickle

In [58]:
with open("second_layer_preds.pkl", "wb") as f: 
  pickle.dump(second_layer_preds, f)

with open("third_layer_preds.pkl", "wb") as f: 
  pickle.dump(third_layer_preds, f)

with open("fourth_layer_preds.pkl", "wb") as f: 
  pickle.dump(fourth_layer_preds, f)

In [69]:
model.save(MODEL_FNAME)

In [70]:
from keras.models import load_model
model = load_model("/content/work/1642622595.826117/model.h5")

In [74]:
!zip -r work.zip work > /dev/null